In [84]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.utils import plot_model

In [85]:
df = pd.read_csv('uci-news-aggregator.csv')

In [86]:
# Clean the headlines
def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower().strip()  # Convert to lowercase and remove leading/trailing spaces
    return text

df['clean_title'] = df['TITLE'].apply(clean_text)

In [87]:
# Split the data into training and testing sets
X = df['clean_title']
y = df['CATEGORY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
# Encode the labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [89]:
vocab_size = 5000
max_length = 5

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [90]:
embedding_dim = 16

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(64, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy', Recall(), Precision()])

model.fit(X_train_pad, y_train, epochs=10, batch_size=32)

Epoch 1/10
10561/10561 [==============================] - 35s 3ms/step - loss: -172.6852 - accuracy: 0.4860 - recall_2: 0.9808 - precision_2: 0.8330
Epoch 2/10
10561/10561 [==============================] - 32s 3ms/step - loss: -511.4729 - accuracy: 0.5025 - recall_2: 0.9778 - precision_2: 0.8512
Epoch 3/10
10561/10561 [==============================] - 32s 3ms/step - loss: -852.4367 - accuracy: 0.5017 - recall_2: 0.9768 - precision_2: 0.8510
Epoch 4/10
10561/10561 [==============================] - 34s 3ms/step - loss: -1196.3899 - accuracy: 0.5042 - recall_2: 0.9761 - precision_2: 0.8538
Epoch 5/10
10561/10561 [==============================] - 39s 4ms/step - loss: -1545.4139 - accuracy: 0.5063 - recall_2: 0.9746 - precision_2: 0.8566
Epoch 6/10
10561/10561 [==============================] - 39s 4ms/step - loss: -1896.6658 - accuracy: 0.5071 - recall_2: 0.9736 - precision_2: 0.8579
Epoch 7/10
10561/10561 [==============================] - 37s 3ms/step - loss: -2247.3579 - accuracy: 0

In [91]:
loss, accuracy, recall, precision = model.evaluate(X_test_pad, y_test)
f1 = 2 * (precision * recall) / (precision + recall)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1-score: {f1}")

2641/2641 [==============================] - 5s 2ms/step - loss: -3391.4844 - accuracy: 0.4993 - recall_2: 0.9768 - precision_2: 0.8465
Accuracy: 0.49932530522346497
Recall: 0.9768298864364624
Precision: 0.8464823961257935
F1-score: 0.9069969097296315


In [92]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
